#### **Var definitions**
Change it whenever necessary

In [59]:
access_oauth_token = "token <PLACE_HOLDER>" #TODO: delete before commiting the code
jira_url = "https://issues.apache.org/jira/"
project_name = "LANG"
repo_name = "commons-lang"
proj = repo_name
base_path = '/Users/lorenapacheco/Concordia/Masters/'
out = base_path + "BugReportsMining/bug-reports/"
proj_path = base_path + "open_source_repos_being_studied/" + repo_name
out_commits_dir = base_path + "BugReportsMining/commits/" + repo_name

#### **Bug reports mining**
Getting all the **Jira tickets** ids from the project that are **closed** and have the **bug issue type**

In [55]:
import os
import json
import requests
import pprint
from jira import JIRA

issue_numbers_list = []

def get_jira_closed_bug_issues(jira_url, project_name):
    issue_search_str = "project = "+ project_name + " AND issuetype = Bug AND status = Closed AND resolution in (Fixed, Done, Resolved) ORDER BY priority DESC, updated DESC"
    issues = jira.search_issues(issue_search_str, maxResults=False)
    for i in issues:
        issue_numbers_list.append(str(i))
    print(issue_numbers_list)
    print("\nNumber of issues: " + str(len(issue_numbers_list)))

jira = JIRA(server=jira_url)
get_jira_closed_bug_issues(jira_url, project_name)

['LANG-1245', 'LANG-987', 'LANG-45', 'LANG-805', 'LANG-421', 'LANG-20', 'LANG-1419', 'LANG-1292', 'LANG-1147', 'LANG-1003', 'LANG-759', 'LANG-549', 'LANG-262', 'LANG-42', 'LANG-26', 'LANG-123', 'LANG-1667', 'LANG-1669', 'LANG-1630', 'LANG-1613', 'LANG-1596', 'LANG-1543', 'LANG-1395', 'LANG-1396', 'LANG-1397', 'LANG-1060', 'LANG-1384', 'LANG-1364', 'LANG-1361', 'LANG-1348', 'LANG-1349', 'LANG-1350', 'LANG-1355', 'LANG-1360', 'LANG-1357', 'LANG-1287', 'LANG-1265', 'LANG-1320', 'LANG-1285', 'LANG-1310', 'LANG-1276', 'LANG-1144', 'LANG-1319', 'LANG-1214', 'LANG-1199', 'LANG-1286', 'LANG-1216', 'LANG-1202', 'LANG-1131', 'LANG-1018', 'LANG-1192', 'LANG-1252', 'LANG-1261', 'LANG-1122', 'LANG-1194', 'LANG-1132', 'LANG-1123', 'LANG-1190', 'LANG-1114', 'LANG-1248', 'LANG-1120', 'LANG-1128', 'LANG-1251', 'LANG-1141', 'LANG-1219', 'LANG-1178', 'LANG-1040', 'LANG-1226', 'LANG-1230', 'LANG-1116', 'LANG-1038', 'LANG-1205', 'LANG-1126', 'LANG-1232', 'LANG-1162', 'LANG-1163', 'LANG-1262', 'LANG-1193', 

#### **Bug reports content extraction**
Extraction the content of each of these bug reports

In [62]:
output_folder = out + repo_name+ '/'

def get_bug_comments(issue):
    comments = issue.fields.comment.comments
    comments_list = []
    for c in comments:
        comment = {
            "author": {
                "login": c.author.key
            },
            "authorAssociation": "NONE",
            "body": c.body,
            "createdAt": c.created,
            "includesCreatedEdit": False,
            "isMinimized": False,
            "minimizedReason": "",
            "reactionGroups": []
        }
        comments_list.append(comment)
    return comments_list

def get_bug_details(issue_number):
    issue = jira.issue(issue_number)
    bug_details = {
        "title": issue.fields.summary,
        "body": issue.fields.description,
        "comments_content": get_bug_comments(issue),
        "createdAt": issue.fields.created,
        "id": issue_number,
        "labels":[
            {
                "name": issue.fields.issuetype.name,
                "id": issue.fields.issuetype.id,
                "description": issue.fields.issuetype.description,
                "color": None
            }
        ],
        "number": issue_number,
        "state": issue.fields.status.name,
        "updatedAt": issue.fields.updated,
        "url": jira_url + "browse/"+ issue_number
    }
    return bug_details
    

def dict_to_json_file(file, dic, folder=output_folder):
    if not os.path.exists(folder):
        os.makedirs(folder)
    with open(os.path.join(folder, file+'.json'), 'w') as fp:
        json.dump(dic, fp, sort_keys=True, indent=4)
    fp.close()
        
for issue_number in issue_numbers_list:
    bug_details = get_bug_details(issue_number)
    dict_to_json_file(repo_name+'-'+str(issue_number), bug_details)
print ("Extraction completed")
    

Extraction completed


#### **Filtering for logs and stack traces**
Filtering only the bug reports with **logs** and **stack traces**

In [63]:
import re
import glob
from collections import defaultdict

# global dict of a dict for storing the log
bug_report_log = defaultdict(dict)
bug_report_stack_trace = defaultdict(dict)

def json_file_to_dict(file):
    data = {}
    with open(os.path.join(output_folder, file+'.json'), 'r') as fp:
        data = json.load(fp)
    fp.close()
    return data

def find_regex(regex, var,file_name, text_content):
    results = re.finditer(regex, text_content)
    if results:
        for log in results:
            bug_name = file_name.split('-')[-2] + '-' + file_name.split('-')[-1]
            var[proj][bug_name] = log.group() 
            
            
def find_logs_and_stack_traces (file_name, text_content):
    stack_trace_regex = r'(?m)^.*?Exception.*(?:\n+^\s*at .*)+'
    logs_regex = r'(ERROR|INFO|WARN|DEBUG|FATAL)\s+(?P<class>\w+(\.\w+)*)'
    find_regex(logs_regex, bug_report_log,file_name, text_content)
    find_regex(stack_trace_regex, bug_report_stack_trace,file_name, text_content)
    

# for every .json in bug-reports folder
for file in glob.glob(output_folder +'/*.json'):
    file_name = os.path.basename(file).replace('.json', '')
    bug_report = json_file_to_dict(file.replace('.json', ''))
    if not bug_report:
        continue
    
    title = bug_report['title'] if bug_report['title'] else ""
    description = bug_report['body'] if bug_report['body'] else ""
    text_content = title + '\n' + description + '\n' + '\n'.join([comment['body'] for comment in bug_report['comments_content'] if 'body' in comment and comment['body']])
    find_logs_and_stack_traces(file_name, text_content)

print(proj)
print("Bug reports with logs:" + str(len(bug_report_log[proj])))
print("Bug reports with stack traces:" + str(len(bug_report_stack_trace[proj])))

commons-lang
Bug reports with logs:3
Bug reports with stack traces:27


#### **Commits search**
Searching for the respective commits

**TODO:** Figure out a way to decide witch of the commits to use

In [64]:
from urllib.parse import quote

bug_reports_with_logs_commits = defaultdict(dict)
bug_reports_with_stack_traces_commits = defaultdict(dict)
cont = 0

def get_bug_report_commit(bug, proj_path):
    os.chdir(proj_path)
    log_grep_command = "git log --grep=\""+ str(bug)+"[^0-9]\" --grep=\""+ str(bug)+"$\" --pretty=format:\"%H\" >> commit_output"
    os.system(log_grep_command)
    lines = []
    with open("commit_output", 'r') as fp:
        lines =fp.readlines()
        fp.close()
    os.system("rm commit_output")
    if (lines):
        commit_hash = lines[0]
        return(commit_hash)
    return ""

for bug in bug_report_log[proj].keys():
    commit = get_bug_report_commit(bug, proj_path)
    if (commit != ""):
        content = {
            "log": bug_report_log[proj][bug],
            "commit": commit.strip()
        }
        bug_reports_with_logs_commits[bug] = content
        cont = cont + 1

for bug in bug_report_stack_trace[proj].keys():
    commit = get_bug_report_commit(bug, proj_path)
    if (commit != ""):
        content = {
            "stack_trace": bug_report_stack_trace[proj][bug],
            "commit": commit.strip()
        }
        bug_reports_with_stack_traces_commits[bug] = content
        cont = cont + 1

if bug_reports_with_logs_commits:
    dict_to_json_file("bug_reports_with_logs_commits", bug_reports_with_logs_commits, out_commits_dir)
if bug_reports_with_stack_traces_commits:
    dict_to_json_file("bug_reports_with_stack_traces_commits", bug_reports_with_stack_traces_commits, out_commits_dir)

print("Extraction completed")
print("Number obtained: " + str(cont))

Extraction completed
Number obtained: 24
